In [ ]:
# Kernal dead error in this pipeline, go to below implementation
import os
import glob
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler
import gc

# Paths
eeg_data = '/storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata'
output_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data3'
os.makedirs(output_dir, exist_ok=True)

# Get .vhdr files
all_vhdr_files = glob.glob(os.path.join(eeg_data, '*.vhdr'))

# Filter out specific files
vhdr_paths = [
    f for f in all_vhdr_files
    if 'Move' not in os.path.basename(f)
    and 'session3_sub13' not in os.path.basename(f)
    and 'session1_sub23' not in os.path.basename(f)
    and 'session2_sub7' not in os.path.basename(f)
    and 'session2_sub12' not in os.path.basename(f)
    and 'session2_sub6' not in os.path.basename(f)
]

wrist_events = {'pronation': 91, 'supination': 101}

for i, vhdr_path in enumerate(vhdr_paths):
    try:
        print(f"\nProcessing: {vhdr_path}")

        raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)

        # Drop EOG & EMG channels (artifact channels)
        raw.drop_channels([ch for ch in raw.ch_names if 'EOG' in ch or 'EMG' in ch])

        # Downsample to 256 Hz to reduce memory/computation load
        raw.resample(256)

        # Notch filter at 60 Hz (remove line noise)
        raw.notch_filter(freqs=60)

        # Band-pass filter for MI band (8-30 Hz)
        raw.filter(8., 30., fir_design='firwin')

        # ICA - limit components and fit on cropped data to save memory/time
        ica = mne.preprocessing.ICA(n_components=10, random_state=97, max_iter='auto')
        ica.fit(raw.copy().crop(tmax=60))  # Use first 60 sec for ICA fitting

        # Automatically find components to exclude - e.g. EOG artifacts if EOG present
        # (You can customize or manually inspect)
        # eog_indices, eog_scores = ica.find_bads_eog(raw)
        # ica.exclude = eog_indices

        # ICA to remove artifacts
        ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter='auto')
        ica.fit(raw)
        ica.exclude = []  # Optional: you can set based on find_bads_* methods
        raw = ica.apply(raw)

        # Extract events and epochs for pronation/supination
        events, event_id = mne.events_from_annotations(raw)

        epochs = mne.Epochs(
            raw, events, event_id=wrist_events,
            tmin=-0.2, tmax=2.0,
            baseline=(None, 0),
            preload=True
        )

        # Artifact rejection: drop epochs with peak-to-peak amplitude >150 µV
        epochs.drop_bad(reject=dict(eeg=150e-6))

        if len(epochs) == 0:
            print(f"All epochs rejected for {vhdr_path}")
            continue

        # Extract data and labels
        X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
        y = epochs.events[:, -1]
        y = np.array([0 if label == 91 else 1 for label in y])  # Binary labels

        # Standardize each epoch (channel-wise)
        X_scaled = np.zeros_like(X, dtype=np.float32)
        for j in range(X.shape[0]):
            scaler = StandardScaler()
            # transpose to (n_times, n_channels), scale, then transpose back
            X_scaled[j] = scaler.fit_transform(X[j].T).T

        # Save processed data
        vhdr_name = os.path.splitext(os.path.basename(vhdr_path))[0]
        output_path = os.path.join(output_dir, f"{vhdr_name}.npz")
        np.savez_compressed(output_path, X=X_scaled, y=y)

        print(f"Saved: {output_path} (X={X_scaled.shape}, y={y.shape})")

        # Free memory
        del raw, events, event_id, epochs, X, y, X_scaled, ica
        gc.collect()

    except Exception as e:
        print(f"Failed processing {vhdr_path}: {e}")



Processing: /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr
Extracting parameters from /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr...
Setting channel info structure...
Reading 0 ... 2564199  =      0.000 ...  1025.680 secs...


/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


Fitting ICA to data using 60 channels (please be patient, this may take a while)


...

/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_24872/1726901494.py:33: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 423 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.6s finished


In [ ]:
import os
import glob
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler
import gc

# Paths
eeg_data = '/storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata'
output_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data3'
os.makedirs(output_dir, exist_ok=True)

# File list
all_vhdr_files = glob.glob(os.path.join(eeg_data, '*.vhdr'))

# Exclude corrupted or low-quality files
skip_files = ['session3_sub13', 'session1_sub23', 'session2_sub7', 'session2_sub12', 'session2_sub6']
vhdr_paths = [f for f in all_vhdr_files if 'Move' not in os.path.basename(f) and all(skip not in f for skip in skip_files)]

# Class mapping
wrist_events = {'pronation': 91, 'supination': 101}

for vhdr_path in vhdr_paths:
    try:
        print(f"\nProcessing: {vhdr_path}")
        raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)

        # Drop EOG & EMG channels
        raw.drop_channels([ch for ch in raw.ch_names if 'EOG' in ch or 'EMG' in ch])

        # Downsample
        raw.resample(256, npad="auto")

        # Filter
        raw.notch_filter(freqs=[60])
        raw.filter(8., 30., fir_design='firwin')

        # ICA
        ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter='auto')
        ica.fit(raw)
        raw = ica.apply(raw)

        # Events
        events, event_id = mne.events_from_annotations(raw)
        epochs = mne.Epochs(
            raw, events, event_id=wrist_events,
            tmin=-0.2, tmax=2.0,
            baseline=(None, 0),
            preload=True,
            reject=dict(eeg=150e-6)
        )

        if len(epochs) == 0:
            print(f"All epochs rejected for {vhdr_path}")
            continue

        # Data & labels
        X = epochs.get_data().astype(np.float32)  # (n_epochs, n_channels, n_times)
        y = np.array([0 if ev[-1] == 91 else 1 for ev in epochs.events], dtype=np.int8)

        # Standardize (channel-wise per epoch)
        for i in range(X.shape[0]):
            X[i] = StandardScaler().fit_transform(X[i].T).T

        # Save
        filename = os.path.splitext(os.path.basename(vhdr_path))[0]
        np.savez_compressed(os.path.join(output_dir, f"{filename}.npz"), X=X, y=y)

        print(f"Saved: {filename}.npz | Shape: {X.shape}")

        # Cleanup
        del raw, epochs, X, y, ica
        gc.collect()

    except Exception as e:
        print(f"Error processing {vhdr_path}: {e}")



Processing: /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr
Extracting parameters from /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr...
Setting channel info structure...
Reading 0 ... 2564199  =      0.000 ...  1025.680 secs...


/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 16501 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 4125 samples (1.650 s)



...

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Fitting ICA to data using 60 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.7s finished


Selecting by number: 15 components
Fitting ICA took 28.2s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 60 PCA components
Used Annotations descriptions: [np.str_('Comment/actiCAP Active Shield On'), np.str_('New Segment/'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S  9'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 13'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 91'), np.str_('Stimulus/S101')]
Not setting metadata
100 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 5501 original time points ...
    Rejecting  epoch based on EEG : ['FT8']
1 bad epochs dropped
Saved: session2_sub3_twist_MI.npz | Shape: (99, 60, 5501)

Processing: /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session1_sub17_twist_MI.vhdr
Extracting parameters from /storage/projects1/e19-

/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26378/2251272052.py:26: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


In [1]:
import os
import glob
import numpy as np
import mne
import gc
import time
from sklearn.preprocessing import StandardScaler


# ---------- Paths ----------
eeg_data = '/storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata'
output_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data3'
os.makedirs(output_dir, exist_ok=True)

# ---------- Files to skip ----------
skip_files = ['session3_sub13', 'session1_sub23', 'session2_sub7', 'session2_sub12', 'session2_sub6']
all_vhdr_files = glob.glob(os.path.join(eeg_data, '*.vhdr'))
vhdr_paths = [f for f in all_vhdr_files if 'Move' not in os.path.basename(f) and all(skip not in f for skip in skip_files)]

# ---------- Class mapping ----------
wrist_events = {'pronation': 91, 'supination': 101}

# ---------- Process each file ----------
for vhdr_path in vhdr_paths:
    try:
        print(f"\n🧠 Processing: {vhdr_path}")
        raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)

        # Drop EOG & EMG channels
        raw.drop_channels([ch for ch in raw.ch_names if 'EOG' in ch or 'EMG' in ch])

        # Crop to reduce memory usage (optional: adjust tmax based on your needs)
        # raw.crop(tmin=0, tmax=180)  # First 3 minutes only

        # Downsample early to save memory
        raw.resample(256, npad="auto")

        # Filter
        raw.notch_filter(freqs=[60])
        raw.filter(8., 30., fir_design='firwin')

        # ICA on cropped data for efficiency
        ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter='auto')
        ica.fit(raw.copy().crop(tmax=60))  # Only first 60s for fitting
        raw = ica.apply(raw)

        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Epoching
        epochs = mne.Epochs(
            raw, events, event_id=wrist_events,
            tmin=-0.2, tmax=2.0,
            baseline=(None, 0),
            preload=True,
            reject=dict(eeg=150e-6)  # Remove high amplitude artifacts
        )

        if len(epochs) == 0:
            print(f"⚠️ All epochs rejected for {vhdr_path}")
            continue

        # Extract data and labels
        X = epochs.get_data().astype(np.float32)  # (n_epochs, n_channels, n_times)
        y = np.array([0 if ev[-1] == 91 else 1 for ev in epochs.events], dtype=np.int8)

        # Standardize channel-wise
        for i in range(X.shape[0]):
            X[i] = StandardScaler().fit_transform(X[i].T).T

        # Save
        filename = os.path.splitext(os.path.basename(vhdr_path))[0]
        np.savez_compressed(os.path.join(output_dir, f"{filename}.npz"), X=X, y=y)

        print(f"✅ Saved: {filename}.npz | Shape: {X.shape}")

        # Cleanup
        del raw, epochs, X, y, ica
        gc.collect()
        time.sleep(1)  # Allow time for memory to clean up

    except Exception as e:
        print(f"❌ Error processing {vhdr_path}: {e}")



🧠 Processing: /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr
Extracting parameters from /storage/projects1/e19-4yp-mi-eeg-for-bci/rawdata/session2_sub14_twist_MI.vhdr...
Setting channel info structure...
Reading 0 ... 2564199  =      0.000 ...  1025.680 secs...


/tmp/ipykernel_26682/98459583.py:27: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26682/98459583.py:27: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
/tmp/ipykernel_26682/98459583.py:27: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (1250.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


Fitting ICA to data using 60 channels (please be patient, this may take a while)


...

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


Fitting ICA to data using 60 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


Selecting by number: 15 components
Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 60 PCA components
Used Annotations descriptions: [np.str_('Comment/actiCAP Active Shield On'), np.str_('New Segment/'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S  9'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 13'), np.str_('Stimulus/S 14'), np.str_('Stimulus/S 91'), np.str_('Stimulus/S101')]
Not setting metadata
100 matching events found
Setting baseline interval to [-0.19921875, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 564 original time points ...
    Rejecting  epoch based on EEG : ['FC6', 'C4', 'C6']
1 bad epochs dropped
✅ Saved: session2_sub17_twist_MI.npz | Shape: (99, 60, 564)
